
<a href="https://colab.research.google.com/github/kokchun/Python-course-AI22/blob/main/Lectures/Lec16-OOP_theoretical.ipynb" target="_parent"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> &nbsp; for interacting with the code

---
# Lecture notes - OOP theoretical

---
This is the lecture note for **OOP inheritance** - but it's built upon contents from previous lectures such as: 
- input-output
- variables
- if-statement
- for loop
- while 
- lists
- random
- strings
- functions
- error handling
- file handling
- dictionary
- OOP

<p class = "alert alert-info" role="alert"><b>Note</b> that this lecture note gives some theoretical depth to Python OOP, but it's not covering everything. Please read more for in depth understanding.


Read more 

- [Classes - Python docs](https://docs.python.org/3/tutorial/classes.html)
---

## Class instantiation process

A class is itself an object of type type. Each class in Python is a callable which means we can call it using ()
When we call the class constructor to create a new instance object of a class the following happens

1. Creating a new instance of the class (`__new__()` is run)
2. The instance is initialized (i.e. `__init__()` is run) giving the instance object its initial state

- [Class instantiation - Real Python](https://realpython.com/python-class-constructor/)

In [1]:
# a class Student is an object itself of type type
class Student:
    pass

# we call the class Student's constructor using the callable syntax
s1 = Student()
print(type(s1))

# instantiate another instance object s2
s2 = Student()

# s1 and s2 is an instance of Student
print(isinstance(s1, Student))
print(isinstance(s2, Student))

# memory address of s1 and s2, note that they are different objects, and hence have different memory addresses
print(hex(id(s1)))
print(hex(id(s2)))

# Student object at memory address of s1
s1

<class '__main__.Student'>
True
True
0x1124bca90
0x1124bcfa0


---
## Attributes 

Attributes of a class are state that holds data and methods for functionality or behavior. Attributes can belong to the class itself, or be instance attributes. The state is basically the values of the variables.

- attributes can be created in the class 
- attributes can be created inside methods
- attributes can be created on the fly using the dot notation .

In [ ]:
s1.name = "Ada"
print(s1.name)

# s1 has the name attribute
print(s1.__dict__)
# note that Student doesn't have name attribute
print(Student.__dict__)

# s2 doesn't have name attribute
print(s2.__dict__)

# this gives an exception
try:
    print(s2.name)
except AttributeError as err:
    print(err)


### Methods 

The behavior of a class constitutes of their methods, which are also attributes of the class. 

In [ ]:
class Student: 
    def __init__(self, name) -> None:
        self.name = name

    def study(self):
        return(f"{self.name} is studying")

# this is an attribute of the class which is a plain function as its not bound to an object
print(Student.study)

# we see that we miss the positional argument self
try: 
    print(Student.study())
except TypeError as err:
    print(err)

# this however is bound to the instance object of type Student, it is a method
# as a method the first parameter is the object itself that is injected, which in turn has the attribute name
print(Student("Ada").study)
print(Student("Ada").study())


In [ ]:
# this is same as passing in the instance object explicitly to the class function
student = Student("Ada")
print(Student.study(student))

---
## Namespaces

Class attributes lives in the class namespace

Namespace - a dictionary of symbolic names (keys): references to objects (values)

Python will first search the local scope which is the innermost score of a function (or method) and then it will go on to the enclosing scope which the function or method reside. In a class this scope is the class namespace and then the interpreter goes to look in the global scope and finally the built-in scope.

- [Python namespace scopes - Real Python](https://realpython.com/python-namespaces-scope/)
- [Special methods](https://www.pythonlikeyoumeanit.com/Module4_OOP/Special_Methods.html)

In [ ]:
class Rabbit:
    # class attributes
    # they exist in the class namespace, not in the instance namespace
    eyes = 2
    nose = 1
    has_tail = True

    # ------------ behaviors (methods) -----------
    # technically they are functions of the class, they become methods first when an instance object is instantiated from the class 
    # then they are methods, which are functions bound to the instance - this in turn means that the instance object itself is passed to 
    # the method as its first parameter. Then the method can access the attributes that exist in the instance itself.

    # dunder init - special method or dunder methods, which will affect the object when an instance is created the init will run to initialize that 
    # instance
    def __init__(self, name) -> None:
        self.name = name

    def binky(self) -> None:
        print(f"{self.name} är glad, hoppar runt och gör binkies")

# an instance object of symbol rabbit1 is instantiated from the class Rabbit
# it 
rabbit1 = Rabbit("Bella")
rabbit2 = Rabbit("Skutt")

# an instance of Rabbit has type Rabbit
print(type(rabbit1))

# we see that in rabbits instances only has name in its namespace (the local namespace)
rabbit1.__dict__, rabbit2.__dict__


### Class scope (enclosing namespace)

In [ ]:
try:
    # the Rabbit object doesn't have an attribute name, as only the instances of the Rabbit class has it 
    # this is due to __init__ hasn't ran yet, which only runs when an instance object is created  
    print(Rabbit.name)
except AttributeError as err:
    print(err)

# this works because an instance of Rabbit is created and the name attribute is initialized after __init__ has run 
Rabbit("Bella").name

In [ ]:
rabbit1 = Rabbit("Bella")
rabbit1.__dict__

In [ ]:
# the name attribute exists in the instance namespace and not in the class namespace
Rabbit("Bella").__dict__

In [ ]:
# note that this works as Python interpreter looks for eyes amd has_tail attributes in the instance but can't find it
# then it will look at its enclosing scope which is the Rabbit class and finds those attributes and retrieves it
rabbit1.eyes, rabbit1.has_tail

In [ ]:
# if we change an instance attribute it doesn't change its class attribute
# for example remove the tail from rabbit1
rabbit1.has_tail = False
rabbit1.__dict__

In [ ]:
rabbit2.__dict__ # doesn't have has_tail

In [ ]:
# has_tail is still True here since only an instance attribute has been changed
Rabbit.__dict__

---
## Property

Attributes that are directly accessed from outside of the class is called bare attributes. Directly accessing some bare attributes are discouraged in times as you want to have more control of the attribute, e.g. to compute a value and/or to have error handling.

This can be done through properties where the bare attribute is private (pseudoprivate, private by convention in Python), and the attribute value itself is provided through the property interface. Many other languages uses getters and setters to provide this functionality, but in Python we have a property function which adds functionality to an existing attribute.

### Getters and setters

To provide interface to an attribute 

In [ ]:
# in other languages

class Square:
    def __init__(self, side) -> None:
        # make the attribute private by convention
        self._side = side

    def get_side(self):
        return self._side
    
    # the setter has important error handling
    def set_side(self, value):
        if not isinstance(value, (int, float)) or value < 0:
            raise ValueError(f"Value must be non-negative number (int or float), not {type(value).__name__}")
        self._side = value
    

square = Square(10)

# get_side is a bound method
print(square.get_side)
# need to call it using callable syntax
print(square.get_side())

# this syntax using get and set, is common in other languages but not Pythonic as it isn't as legible
# as just calling the bare attribute, e.g. square.side, square.side = 20
square.set_side(20)
print(square.get_side())

### Property

We continue with getters and setters but, we add functionality to those methods using property function.

This gives a clean syntax for accessing the attribute

In [ ]:
class Square:
    def __init__(self, side) -> None:
        self._side = side

    def get_side(self):
        return self._side

    def set_side(self, value):
        if not isinstance(value, (int, float)) or value < 0:
            raise ValueError(
                f"Value must be non-negative number (int or float), not {type(value).__name__}"
            )
        self._side = value

    side = property(fget=get_side, fset=set_side)

square = Square(10)

# now we can access it as we would access a bare attribute
print(square.side)

# also we can mutate it as we would a bare attribute, but we get the error checking
square.side = 20
print(square.side)

# error checking
try:
    square.side = -2
except ValueError as err:
    print(err)

### Property with decorator syntax

In [3]:
class Square:
    def __init__(self, side) -> None:
        self._side = side

    @property
    def side(self):
        return self._side

    @side.setter
    def side(self, value):
        if not isinstance(value, (int, float)) or value < 0:
            raise ValueError(
                f"Value must be non-negative number (int or float), not {type(value).__name__}"
            )
        self._side = value

    # this is a computed property
    @property
    def area(self):
        return self.side**2

square = Square(10)

# now we can access it as we would access a bare attribute
print(square.side)

# also we can mutate it as we would a bare attribute, but we get the error checking
square.side = 20
print(square.side)

# error checking
try:
    square.side = -2
except ValueError as err:
    print(err)

square.area

10
20
Value must be non-negative number (int or float), not int


400

---

Kokchun Giang

[LinkedIn][linkedIn_kokchun]

[GitHub portfolio][github_portfolio]

[linkedIn_kokchun]: https://www.linkedin.com/in/kokchungiang/
[github_portfolio]: https://github.com/kokchun/Portfolio-Kokchun-Giang

---